In [243]:
import pandas as pd
import pickle as pkl
import os

In [244]:
picklejar = ['cb_1', 'cb_2', 'cb_3', 'cb_4', 'cb_5', 'cb_6', 'cb_7', 'cb_8', 'cb_9', 'cb_10']

#### Full Year Consolidated Data *(RETIRED FOR BEING DUPLICATE OF POPULATION DATA)

In [87]:
#for loop to bring in 10 pickles, drop last 2 rows and last 3 columns of each df, & add column as variable key
fcd_cb = {}

for pickle in picklejar:
    key = pickle
    with open(f'../data/fcd_cbpickles/{pickle}.pickle', 'rb') as handle:
        table = pkl.load(handle)
    for k, value in table.items(): 
        table[k] = value.iloc[:-3]
        table[k] = table[k][['VALUE']]
        table[k]['varnum'] = k
    # fcd_cb[key] = table

##### Next Steps

- turn the dictionary of tiny tables into df

In [88]:
#loop to concat dict of dicts of tables into dict of dfs

for var in fcd_cb:
    df = pd.concat(fcd_cb[var].values(), ignore_index=True)
    fcd_cb[var] = df
    

In [89]:
# fcd_cb['cb_1']

,VALUE,varnum
0,-1 INAPPLICABLE,var_0
1,1 YES,var_0
2,-8 DK,var_1
3,-7 REFUSED,var_1
4,-1 INAPPLICABLE,var_1
...,...,...
6857,1 LESS THAN 1 YEAR,var_1487
6858,"2 1 YEAR, LESS THAN 5 YEARS",var_1487
6859,"3 5 YEARS, LESS THAN 10 YEARS",var_1487
6860,"4 10 YEARS, LESS THAN 15 YEARS",var_1487


In [109]:
#define function to run loops

# def depickle(CB):
#     picklejar = ['cb_1', 'cb_2', 'cb_3', 'cb_4', 'cb_5', 'cb_6', 'cb_7', 'cb_8', 'cb_9', 'cb_10']
    
#     for pickle in picklejar:
#         key = pickle
#         with open(f'../data/{name}pickles/{pickle}.pickle', 'rb') as handle:
#             table = pkl.load(handle)
#         for k, value in table.items(): 
#             table[k] = value.iloc[:-3]
#             table[k] = table[k][['VALUE']]
#             table[k]['varnum'] = k
#         CB[key] = table
#     return CB

In [110]:
# fpc_cb = {}
# CB = fpc_cb
# name = 'fpc_cb'

# fpcCBs = depickle(CB)

In [ ]:
# fpcCBs['cb_1']

it hated that, nevermind

- don't forget to split value column on first space

In [113]:
#read in variable list csvs
#export variables and descriptions to csvs
# fcd_vars = {}

# for cb in picklejar:
#     df = pd.read_csv(f'../data/fcdCBcsvs/{cb}.csv')
#     key = cb
#     fcd_vars[key] = df

In [ ]:
#this is wrong but it doesn't matter bc fpc is apparently identical to fcd, so if that one exported right i can just drop fcd
# fcd_vars['cb_1']

## **FCD OFFICIALLY RETIRED FROM THIS POINT FORWARD**

### *Population Characteristics*

In [155]:
#for loop to bring in 10 pickles, drop last 2 rows and last 3 columns of each df, & add column as variable key
fpc_cb = {}

for pickle in picklejar:
    key = pickle
    with open(f'../data/fpc_cbpickles/{pickle}.pickle', 'rb') as handle:
        table = pkl.load(handle)
    for k, value in table.items(): 
        table[k] = value.iloc[:-3]
        table[k] = table[k][['VALUE']]
        table[k]['varnum'] = k
    fpc_cb[key] = table

In [156]:
#loop to concat dict of dicts of tables into dict of dfs

for var in fpc_cb:
    df = pd.concat(fpc_cb[var].values(), ignore_index=True)
    fpc_cb[var] = df
    

In [159]:
#split value column into value and response
for key in fpc_cb:
    cb = fpc_cb[key]
    cb[['VALUE', 'response']] = cb['VALUE'].str.split(' ',expand=True, n= 1)
    cb = cb.rename(columns= {'VALUE':'value'})
    cb = cb[['varnum', 'value', 'response']]
    fpc_cb[key] = cb

In [186]:
#can i make this a function?

def splitCols(dict):
    for key in dict:
        cb = dict[key]
        cb[['VALUE', 'response']] = cb['VALUE'].str.split(' ',expand=True, n= 1)
        cb = cb.rename(columns= {'VALUE':'value'})
        cb = cb[['varnum', 'value', 'response']]
        dict[key] = cb
    return dict

In [162]:
fpc_cb['cb_1']

,varnum,value,response
0,var_0,-1,INAPPLICABLE
1,var_0,1,YES
2,var_1,-8,DK
3,var_1,-7,REFUSED
4,var_1,-1,INAPPLICABLE
...,...,...,...
6857,var_1487,1,LESS THAN 1 YEAR
6858,var_1487,2,"1 YEAR, LESS THAN 5 YEARS"
6859,var_1487,3,"5 YEARS, LESS THAN 10 YEARS"
6860,var_1487,4,"10 YEARS, LESS THAN 15 YEARS"


In [172]:
#read in variable list csvs, without bottom two rows of trash
fpc_vars = {}

for cb in picklejar:
    df = pd.read_csv(f'../data/fpcCBcsvs/{cb}.csv')
    key = cb
    fpc_vars[key] = df.iloc[:-2]

In [173]:
#add varnum column to act as merge key, and lowercase the column names
for key, value in fpc_vars.items():
    value['varnum'] = [f'var_{i}' for i in range(len(value))]
    value = value.rename(columns= {'Name':'name', 'Description':'description'})
    fpc_vars[key] = value

/var/folders/f_/3c498dtd4q9cs_ll7zmc_9hw0000gn/T/ipykernel_39049/3575195688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value['varnum'] = [f'var_{i}' for i in range(len(value))]


In [191]:
#make this a function too
def addVarnum(dict):
    for key, value in dict.items():
        value['varnum'] = [f'var_{i}' for i in range(len(value))]
        value = value.rename(columns= {'Name':'name', 'Description':'description'})
        dict[key] = value
    return dict

In [174]:
fpc_vars['cb_1']

,name,description,varnum
0,ACCELI42,PERS ELIGIBLE FOR ACCESS SUPPLEMENT-R4/2,var_0
1,ACTDTY31,MILITARY FULL-TIME ACTIVE DUTY - R3/1,var_1
2,ACTDTY42,MILITARY FULL-TIME ACTIVE DUTY - R4/2,var_2
3,ACTDTY53,MILITARY FULL-TIME ACTIVE DUTY - R5/3,var_3
4,ACTLIM31,ANY LIMITATION WORK/HOUSEWRK/SCHL - RD 3/1,var_4
...,...,...,...
1483,WRKLIM53,WORK LIMITATION - RD 5/3,var_1483
1484,YCHJ3142,WHY CHNGD JOB BETWEEN RD 3/1 AND RD 4/2,var_1484
1485,YCHJ4253,WHY CHNGD JOB BETWEEN RD 4/2 AND RD 5/3,var_1485
1486,YNOUSC42_M18,MAIN REAS PERS DOESNT HAVE USC-R4/2,var_1486


In [175]:
# merge all variable dfs onto all value dfs
fpcCBmasters = {}

for key in fpc_vars:
    index = key
    if index in fpc_cb:
        A = fpc_vars[key]
        B = fpc_cb[index]
        merged = pd.merge(A, B, on='varnum', how='inner')
        merged = merged.drop(columns=['varnum'])
        fpcCBmasters[index] = merged

In [194]:
# one mo function
# fpcCBmasters = {}
# master = fpcCBmasters
# merger = fpc_cb

def mergeMasters(dict):
    for key in dict:
        index = key
        if index in merger:
            A = dict[key]
            B = merger[index]
            merged = pd.merge(A, B, on='varnum', how='inner')
            merged = merged.drop(columns=['varnum'])
            master[index] = merged
    return master

In [176]:
fpcCBmasters['cb_1']

,name,description,value,response
0,ACCELI42,PERS ELIGIBLE FOR ACCESS SUPPLEMENT-R4/2,-1,INAPPLICABLE
1,ACCELI42,PERS ELIGIBLE FOR ACCESS SUPPLEMENT-R4/2,1,YES
2,ACTDTY31,MILITARY FULL-TIME ACTIVE DUTY - R3/1,-8,DK
3,ACTDTY31,MILITARY FULL-TIME ACTIVE DUTY - R3/1,-7,REFUSED
4,ACTDTY31,MILITARY FULL-TIME ACTIVE DUTY - R3/1,-1,INAPPLICABLE
...,...,...,...,...
6857,YRSINUS,YEARS PERSON LIVED IN THE US,1,LESS THAN 1 YEAR
6858,YRSINUS,YEARS PERSON LIVED IN THE US,2,"1 YEAR, LESS THAN 5 YEARS"
6859,YRSINUS,YEARS PERSON LIVED IN THE US,3,"5 YEARS, LESS THAN 10 YEARS"
6860,YRSINUS,YEARS PERSON LIVED IN THE US,4,"10 YEARS, LESS THAN 15 YEARS"


In [232]:
#function to rename the dfs with their corresponding year
def renameMasters(dict, start_year, prefix):
   
    renamed_dict = {}
    current_year = start_year

    for i in range(1, len(dict) + 1):
        original_key = f'cb_{i}'
        new_key = f'{prefix}_{current_year}'
        renamed_dict[new_key] = dict[original_key]
        current_year -= 1

    return renamed_dict

In [233]:
#call function to rename dfs with the corresponding year
fpcCBmasters = renameMasters(fpcCBmasters, 2021, 'fpc_cb')

In [234]:
fpcCBmasters['fpc_cb_2021']

,name,description,value,response
0,ACCELI42,PERS ELIGIBLE FOR ACCESS SUPPLEMENT-R4/2,-1,INAPPLICABLE
1,ACCELI42,PERS ELIGIBLE FOR ACCESS SUPPLEMENT-R4/2,1,YES
2,ACTDTY31,MILITARY FULL-TIME ACTIVE DUTY - R3/1,-8,DK
3,ACTDTY31,MILITARY FULL-TIME ACTIVE DUTY - R3/1,-7,REFUSED
4,ACTDTY31,MILITARY FULL-TIME ACTIVE DUTY - R3/1,-1,INAPPLICABLE
...,...,...,...,...
6857,YRSINUS,YEARS PERSON LIVED IN THE US,1,LESS THAN 1 YEAR
6858,YRSINUS,YEARS PERSON LIVED IN THE US,2,"1 YEAR, LESS THAN 5 YEARS"
6859,YRSINUS,YEARS PERSON LIVED IN THE US,3,"5 YEARS, LESS THAN 10 YEARS"
6860,YRSINUS,YEARS PERSON LIVED IN THE US,4,"10 YEARS, LESS THAN 15 YEARS"


In [240]:
#create folder and export codebooks to csv. one, single csv per year. the whole codebook. incredible
def shipCodebooks(dir, dict, prefix):
    if not os.path.exists(dir):
                os.makedirs(dir)
    for cb in dict:
        dict[cb].to_csv(f'../data/{prefix}_codebooks/{cb}.csv', index= False)
    return print('codebooks shipped!')

In [245]:
#call function to create folder and export codebooks to csv
prefix = 'fpc'
dir = f'../data/{prefix}_codebooks'
dict = fpcCBmasters

shipCodebooks(dir, dict, prefix)

codebooks shipped!


### *Hospital Inpatient Stays*

In [187]:
#for loop to bring in 10 pickles, drop last 2 rows and last 3 columns of each df, & add column as variable key
his_cb = {}

for pickle in picklejar:
    key = pickle
    with open(f'../data/his_cbpickles/{pickle}.pickle', 'rb') as handle:
        table = pkl.load(handle)
    for k, value in table.items(): 
        table[k] = value.iloc[:-3]
        table[k] = table[k][['VALUE']]
        table[k]['varnum'] = k
    his_cb[key] = table

In [188]:
#loop to concat dict of dicts of tables into dict of dfs

for var in his_cb:
    df = pd.concat(his_cb[var].values(), ignore_index=True)
    his_cb[var] = df
   

In [189]:
#call function to split value column into value and response
his_cb = splitCols(his_cb)

In [190]:
his_cb['cb_1']

,varnum,value,response
0,var_0,-8,DK
1,var_0,-7,REFUSED
2,var_0,1,YES
3,var_0,2,NO
4,var_1,-8,DK
...,...,...,...
181,var_47,-7,REFUSED
182,var_47,1,YES
183,var_47,2,NO
184,var_48,1,- 8 VARPSU


In [123]:
#read in variable list csvs, without bottom two rows of trash
his_vars = {}

for cb in picklejar:
    df = pd.read_csv(f'../data/hisCBcsvs/{cb}.csv')
    key = cb
    his_vars[key] = df.iloc[:-2]

In [192]:
#call function to add varnum column to act as merge key, and lowercase the column names
his_vars = addVarnum(his_vars)

In [193]:
his_vars['cb_1']

,name,description,varnum
0,ANYOPER,ANY OPERATIONS OR SURGERIES PERFORMED,var_0
1,DSCHPMED,MEDICINES PRESCRIBED AT DISCHARGE,var_1
2,DUID,PANEL # + ENCRYPTED DU IDENTIFIER,var_2
3,DUPERSID,PERSON ID (DUID + PID),var_3
4,EMERROOM,DID STAY BEGIN WITH EMERGENCY ROOM VISIT,var_4
5,ERHEVIDX,EVENT ID FOR CORRESPONDING EMER RM VISIT,var_5
6,EVENTRN,EVENT ROUND NUMBER,var_6
7,EVNTIDX,EVENT ID,var_7
8,FFEEIDX,FLAT FEE ID,var_8
9,FFIPTYPE,FLAT FEE BUNDLE,var_9


In [195]:
# call function to merge all variable dfs onto all value dfs
hisCBmasters = {}
master = hisCBmasters
merger = his_cb

hisCBmasters = mergeMasters(his_vars)

In [196]:
hisCBmasters['cb_1']

,name,description,value,response
0,ANYOPER,ANY OPERATIONS OR SURGERIES PERFORMED,-8,DK
1,ANYOPER,ANY OPERATIONS OR SURGERIES PERFORMED,-7,REFUSED
2,ANYOPER,ANY OPERATIONS OR SURGERIES PERFORMED,1,YES
3,ANYOPER,ANY OPERATIONS OR SURGERIES PERFORMED,2,NO
4,DSCHPMED,MEDICINES PRESCRIBED AT DISCHARGE,-8,DK
...,...,...,...,...
181,SPECCOND,HOSPITAL STAY RELATED TO CONDITION,-7,REFUSED
182,SPECCOND,HOSPITAL STAY RELATED TO CONDITION,1,YES
183,SPECCOND,HOSPITAL STAY RELATED TO CONDITION,2,NO
184,VARPSU,"VARIANCE ESTIMATION PSU, 2021",1,- 8 VARPSU


In [235]:
#call function to rename dfs with the corresponding year
hisCBmasters = renameMasters(hisCBmasters, 2021, 'his_cb')
hisCBmasters['his_cb_2021']

,name,description,value,response
0,ANYOPER,ANY OPERATIONS OR SURGERIES PERFORMED,-8,DK
1,ANYOPER,ANY OPERATIONS OR SURGERIES PERFORMED,-7,REFUSED
2,ANYOPER,ANY OPERATIONS OR SURGERIES PERFORMED,1,YES
3,ANYOPER,ANY OPERATIONS OR SURGERIES PERFORMED,2,NO
4,DSCHPMED,MEDICINES PRESCRIBED AT DISCHARGE,-8,DK
...,...,...,...,...
181,SPECCOND,HOSPITAL STAY RELATED TO CONDITION,-7,REFUSED
182,SPECCOND,HOSPITAL STAY RELATED TO CONDITION,1,YES
183,SPECCOND,HOSPITAL STAY RELATED TO CONDITION,2,NO
184,VARPSU,"VARIANCE ESTIMATION PSU, 2021",1,- 8 VARPSU


In [246]:
#call function to create folder and export codebooks to csv
prefix = 'his'
dir = f'../data/{prefix}_codebooks'
dict = hisCBmasters

shipCodebooks(dir, dict, prefix)

codebooks shipped!


### *Emergency Room Visits*

In [213]:
#for loop to bring in 10 pickles, drop last 2 rows and last 3 columns of each df, & add column as variable key
erv_cb = {}

for pickle in picklejar:
    key = pickle
    with open(f'../data/erv_cbpickles/{pickle}.pickle', 'rb') as handle:
        table = pkl.load(handle)
    for k, value in table.items(): 
        table[k] = value.iloc[:-3]
        table[k] = table[k][['VALUE']]
        table[k]['varnum'] = k
    erv_cb[key] = table

In [214]:
#loop to concat dict of dicts of tables into dict of dfs

for var in erv_cb:
    df = pd.concat(erv_cb[var].values(), ignore_index=True)
    erv_cb[var] = df
   

In [215]:
#call function to split value column into value and response
erv_cb = splitCols(erv_cb)

In [216]:
erv_cb['cb_1']

,varnum,value,response
0,var_0,VALID,ID
1,var_1,VALID,ID
2,var_2,-8,DK
3,var_2,-7,REFUSED
4,var_2,1,YES
...,...,...,...
213,var_52,-8,DK
214,var_52,-7,REFUSED
215,var_52,1,YES
216,var_52,2,NO


In [217]:
#read in variable list csvs, without bottom two rows of trash
erv_vars = {}

for cb in picklejar:
    df = pd.read_csv(f'../data/ervCBcsvs/{cb}.csv')
    key = cb
    erv_vars[key] = df.iloc[:-2]

In [218]:
#call function to add varnum column to act as merge key, and lowercase the column names
erv_vars = addVarnum(erv_vars)

/var/folders/f_/3c498dtd4q9cs_ll7zmc_9hw0000gn/T/ipykernel_39049/3623010826.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value['varnum'] = [f'var_{i}' for i in range(len(value))]


In [219]:
erv_vars['cb_1']

,name,description,varnum
0,DUID,PANEL # + ENCRYPTED DU IDENTIFIER,var_0
1,DUPERSID,PERSON ID (DUID + PID),var_1
2,EKG_M18,"THIS VISIT DID P HAVE AN EKG, EEG OR ECG",var_2
3,ERDATEMM,EVENT DATE - MONTH,var_3
4,ERDATEYR,EVENT DATE - YEAR,var_4
5,ERDMD21X,"DOCTOR AMOUNT PAID, MEDICAID (IMPUTED)",var_5
6,ERDMR21X,"DOCTOR AMOUNT PAID, MEDICARE (IMPUTED)",var_6
7,ERDOF21X,"DOCTOR AMOUNT PAID, OTH FEDERAL (IMPUTED)",var_7
8,ERDOT21X,"DOCTOR AMOUNT PAID, OTH INSUR (IMPUTED)",var_8
9,ERDPV21X,"DOCTOR AMOUNT PAID, PRIV INSUR (IMPUTED)",var_9


In [220]:
# call function to merge all variable dfs onto all value dfs
ervCBmasters = {}
master = ervCBmasters
merger = erv_cb

ervCBmasters = mergeMasters(erv_vars)

In [221]:
ervCBmasters['cb_1']

,name,description,value,response
0,DUID,PANEL # + ENCRYPTED DU IDENTIFIER,VALID,ID
1,DUPERSID,PERSON ID (DUID + PID),VALID,ID
2,EKG_M18,"THIS VISIT DID P HAVE AN EKG, EEG OR ECG",-8,DK
3,EKG_M18,"THIS VISIT DID P HAVE AN EKG, EEG OR ECG",-7,REFUSED
4,EKG_M18,"THIS VISIT DID P HAVE AN EKG, EEG OR ECG",1,YES
...,...,...,...,...
213,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,-8,DK
214,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,-7,REFUSED
215,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,1,YES
216,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,2,NO


In [236]:
#call function to rename dfs with the corresponding year
ervCBmasters = renameMasters(ervCBmasters, 2021, 'erv_cb')
ervCBmasters['erv_cb_2021']

,name,description,value,response
0,DUID,PANEL # + ENCRYPTED DU IDENTIFIER,VALID,ID
1,DUPERSID,PERSON ID (DUID + PID),VALID,ID
2,EKG_M18,"THIS VISIT DID P HAVE AN EKG, EEG OR ECG",-8,DK
3,EKG_M18,"THIS VISIT DID P HAVE AN EKG, EEG OR ECG",-7,REFUSED
4,EKG_M18,"THIS VISIT DID P HAVE AN EKG, EEG OR ECG",1,YES
...,...,...,...,...
213,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,-8,DK
214,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,-7,REFUSED
215,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,1,YES
216,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,2,NO


In [247]:
#call function to create folder and export codebooks to csv
prefix = 'erv'
dir = f'../data/{prefix}_codebooks'
dict = ervCBmasters

shipCodebooks(dir, dict, prefix)

codebooks shipped!


### *Office-Based Medical Provider Visits*

In [222]:
#for loop to bring in 10 pickles, drop last 2 rows and last 3 columns of each df, & add column as variable key
obv_cb = {}

for pickle in picklejar:
    key = pickle
    with open(f'../data/obv_cbpickles/{pickle}.pickle', 'rb') as handle:
        table = pkl.load(handle)
    for k, value in table.items(): 
        table[k] = value.iloc[:-3]
        table[k] = table[k][['VALUE']]
        table[k]['varnum'] = k
    obv_cb[key] = table

In [223]:
#loop to concat dict of dicts of tables into dict of dfs

for var in obv_cb:
    df = pd.concat(obv_cb[var].values(), ignore_index=True)
    obv_cb[var] = df
   

In [224]:
#call function to split value column into value and response
obv_cb = splitCols(obv_cb)

In [225]:
obv_cb['cb_1']

,varnum,value,response
0,var_0,-8,DK
1,var_0,-7,REFUSED
2,var_0,-1,INAPPLICABLE
3,var_0,1,YES
4,var_0,2,NO
...,...,...,...
243,var_46,-7,REFUSED
244,var_46,-1,INAPPLICABLE
245,var_46,1,YES
246,var_46,2,NO


In [226]:
#read in variable list csvs, without bottom two rows of trash
obv_vars = {}

for cb in picklejar:
    df = pd.read_csv(f'../data/obvCBcsvs/{cb}.csv')
    key = cb
    obv_vars[key] = df.iloc[:-2]

In [227]:
obv_vars['cb_1']

,Name,Description
0,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV
1,DRSPLTY_M18,MVIS DOCTOR'S SPECIALTY
2,DUID,PANEL # + ENCRYPTED DU IDENTIFIER
3,DUPERSID,PERSON ID (DUID + PID)
4,EKG_M18,"THIS VISIT DID P HAVE AN EKG, EEG OR ECG"
5,EVENTRN,EVENT ROUND NUMBER
6,EVNTIDX,EVENT ID
7,FFBEF21,TOTAL # OF VISITS IN FF BEFORE 2021
8,FFEEIDX,FLAT FEE ID
9,FFOBTYPE,FLAT FEE BUNDLE


In [228]:
#call function to add varnum column to act as merge key, and lowercase the column names
obv_vars = addVarnum(obv_vars)

/var/folders/f_/3c498dtd4q9cs_ll7zmc_9hw0000gn/T/ipykernel_39049/3623010826.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value['varnum'] = [f'var_{i}' for i in range(len(value))]


In [229]:
obv_vars['cb_1']

,name,description,varnum
0,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,var_0
1,DRSPLTY_M18,MVIS DOCTOR'S SPECIALTY,var_1
2,DUID,PANEL # + ENCRYPTED DU IDENTIFIER,var_2
3,DUPERSID,PERSON ID (DUID + PID),var_3
4,EKG_M18,"THIS VISIT DID P HAVE AN EKG, EEG OR ECG",var_4
5,EVENTRN,EVENT ROUND NUMBER,var_5
6,EVNTIDX,EVENT ID,var_6
7,FFBEF21,TOTAL # OF VISITS IN FF BEFORE 2021,var_7
8,FFEEIDX,FLAT FEE ID,var_8
9,FFOBTYPE,FLAT FEE BUNDLE,var_9


In [230]:
# call function to merge all variable dfs onto all value dfs
obvCBmasters = {}
master = obvCBmasters
merger = obv_cb

obvCBmasters = mergeMasters(obv_vars)

In [231]:
obvCBmasters['cb_1']

,name,description,value,response
0,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,-8,DK
1,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,-7,REFUSED
2,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,-1,INAPPLICABLE
3,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,1,YES
4,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,2,NO
...,...,...,...,...
243,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,-7,REFUSED
244,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,-1,INAPPLICABLE
245,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,1,YES
246,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,2,NO


In [237]:
#call function to rename dfs with the corresponding year
obvCBmasters = renameMasters(obvCBmasters, 2021, 'obv_cb')
obvCBmasters['obv_cb_2021']

,name,description,value,response
0,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,-8,DK
1,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,-7,REFUSED
2,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,-1,INAPPLICABLE
3,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,1,YES
4,DOCATLOC,ANY MD WORK AT LOCATION WHERE P SAW PROV,2,NO
...,...,...,...,...
243,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,-7,REFUSED
244,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,-1,INAPPLICABLE
245,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,1,YES
246,XRAYS_M18,THIS VISIT DID P HAVE X-RAYS,2,NO


In [248]:
#call function to create folder and export codebooks to csv
prefix = 'obv'
dir = f'../data/{prefix}_codebooks'
dict = obvCBmasters

shipCodebooks(dir, dict, prefix)

codebooks shipped!
